In [92]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import yaml
from PIL import Image
import numpy as np
import os

In [102]:
# Load YAML file with an absolute path
with open('/home/tyler/yolov5/data/coco128.yaml', 'r') as file:
    data = yaml.safe_load(file)

# Extract the dataset path
dataset_path = data['path']

# Extract image paths and labels for train, val, and test sets
train_image_paths = data.get('train', [])
train_labels = data.get('train_labels', [])  # Adjust the key based on your YAML structure

val_image_paths = data.get('val', [])
val_labels = data.get('val_labels', [])  # Adjust the key based on your YAML structure

test_image_paths = data.get('test', [])
test_labels = data.get('test_labels', [])  # Adjust the key based on your YAML structure

# Define batch size and number of epochs
batch_size = 16
epochs = 10
target_size = (224, 224)

def custom_data_generator(image_paths, labels, batch_size=batch_size, target_size=target_size):
    while True:
        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]

            # Load and preprocess batch images
            batch_images = load_and_preprocess_images(batch_paths, dataset_path, target_size)

            # Assuming batch_labels are already in one-hot encoded format
            batch_labels_encoded = np.array(batch_labels)

            yield batch_images, batch_labels_encoded

def load_and_preprocess_images(paths, dataset_path, target_size=(416, 416)):
    images = []
    for path in paths:
        full_path = os.path.join(dataset_path, path)
        print("Full Path:", full_path)

        try:
            if os.path.isfile(full_path) and not os.path.basename(full_path).startswith('.'):
                # Load image using PIL
                img = Image.open(full_path)

                # Convert image to numpy array
                img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]

                # Resize the image to the target size
                img_array = cv2.resize(img_array, target_size)

                images.append(img_array)

                # Print the shape of the loaded image
                print("Image Shape:", img_array.shape)

        except Exception as e:
            print(f"Error loading image: {full_path}")
            print(f"Exception: {e}")

    return np.array(images)




# Calculate the number of steps per epoch for training
steps_per_epoch = max(1, len(train_image_paths) // batch_size)

# Calculate the number of steps per epoch for validation
validation_steps = max(1, len(val_image_paths) // batch_size)

# Use the custom data generator
custom_train_generator = custom_data_generator(train_image_paths, train_labels, target_size=target_size)
custom_test_generator = custom_data_generator(test_image_paths, test_labels, target_size=target_size)

# Use the custom data generator to create train and test generators
train_generator = custom_data_generator(train_image_paths, train_labels, batch_size=batch_size, target_size=target_size)
test_generator = custom_data_generator(test_image_paths, test_labels, batch_size=batch_size, target_size=target_size)


In [103]:
# Model Architecture (simple image classification)
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(units=80, activation='softmax')  # num_classes is the number of object classes
])

In [104]:
# Compile your model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [105]:
for batch_images, batch_labels_encoded in custom_train_generator:
    print("Batch Shape:", batch_images.shape)
    break  # Print only the first batch for inspection

# Fit the model using the generators
model.fit(custom_train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
          validation_data=custom_test_generator, validation_steps=validation_steps)


Full Path: ../datasets/coco128/i
Full Path: ../datasets/coco128/m
Full Path: ../datasets/coco128/a
Full Path: ../datasets/coco128/g
Full Path: ../datasets/coco128/e
Full Path: ../datasets/coco128/s
Full Path: /
Full Path: ../datasets/coco128/t
Full Path: ../datasets/coco128/r
Full Path: ../datasets/coco128/a
Full Path: ../datasets/coco128/i
Full Path: ../datasets/coco128/n
Full Path: ../datasets/coco128/2
Full Path: ../datasets/coco128/0
Full Path: ../datasets/coco128/1
Full Path: ../datasets/coco128/7
Batch Shape: (0,)
Full Path: ../datasets/coco128/i
Full Path: ../datasets/coco128/m
Full Path: ../datasets/coco128/a
Full Path: ../datasets/coco128/g
Full Path: ../datasets/coco128/e
Full Path: ../datasets/coco128/s
Full Path: /
Full Path: ../datasets/coco128/t
Full Path: ../datasets/coco128/r
Full Path: ../datasets/coco128/a
Full Path: ../datasets/coco128/i
Full Path: ../datasets/coco128/n
Full Path: ../datasets/coco128/2
Full Path: ../datasets/coco128/0
Full Path: ../datasets/coco128/1

ValueError: in user code:

    File "/home/tyler/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/tyler/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/tyler/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/tyler/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/home/tyler/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/tyler/.local/lib/python3.10/site-packages/keras/src/engine/input_spec.py", line 253, in assert_input_compatibili

In [77]:
# Evaluate your model on the test set
test_generator = custom_data_generator(test_image_paths, test_labels, batch_size, target_size)
test_steps = len(test_image_paths) // batch_size
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps)


TypeError: object of type 'NoneType' has no len()

In [ ]:
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Save your model
model.save('custom_object_detection_model.h5')